In [ ]:
import seaborn as sns
import pandas as pd
from googleapiclient.discovery import build
import warnings
warnings.filterwarnings('ignore')

In [ ]:
apiKey = 'Your API KEY'
channelIDList = ["Your channel ID List"]
channelID = ','.join(channelIDList)
youtube = build('youtube','v3',developerKey=apiKey)
print(channelID)

### # function to get youtube channel statistics.

In [ ]:

def getchannelStats(cID):
    dataframe = []
    request = request = youtube.channels().list(
        part='snippet,contentDetails,statistics',
        id=channelID,
    )
    response  = request.execute()
    try:
      for i in range(len(response['items'])):
          data = dict(
          ChannelName = response['items'][i]['snippet']['title'],
          Subscribers = response['items'][i]['statistics']['subscriberCount'],
          Views = response['items'][i]['statistics']['viewCount'],
          Videos = response['items'][i]['statistics']['videoCount'],
          PlaylistID = response['items'][i]['contentDetails']['relatedPlaylists']['uploads'],
          # ChannelID = response['items'][i]['id'],
          Description = response['items'][i]['snippet']['description']
          )
          dataframe.append(data)
      return dataframe
    except Exception as e:
        return f'Some error occured due to {e}'

In [ ]:
archivedata = getchannelStats(channelID)
archive = pd.DataFrame(archivedata)
archive

In [ ]:
archive.dtypes

In [ ]:
archive['Subscribers'] = pd.to_numeric(archive['Subscribers'])
archive['Videos'] = pd.to_numeric(archive['Videos'])
archive['Views'] = pd.to_numeric(archive['Views'])
archive.dtypes

In [ ]:
sns.set(rc={'figure.figsize':(20,10)})
ax = sns.barplot(x='ChannelName', y ='Subscribers', data= archive)

In [ ]:
sns.set(rc={'figure.figsize':(20,5)})
ax = sns.barplot(x='ChannelName', y ='Views', data= archive)

In [ ]:
sns.set(rc={'figure.figsize':(20,10)})
ax = sns.barplot(x='ChannelName', y ='Videos', data= archive)

In [ ]:
archive

In [ ]:
playlistid = archive.loc[archive['ChannelName']=='<Channel Name>','PlaylistID'].iloc[0]
playlistid

### function to get a specific channel's video IDs.

In [ ]:
def getvideoids(playlistid):
  request = youtube.playlistItems().list(
        part='contentDetails',
        playlistId=playlistid,
        maxResults = 50
    )
  response = request.execute()
  videoids = []
  for i in range(len(response['items'])):
        videoids.append(response['items'][i]['contentDetails']['videoId'])

  nextpagetoken = response.get('nextPageToken')
  morepages = True
  while morepages:
    if nextpagetoken is None:
      morepages = False
    else:
      request = youtube.playlistItems().list(
        part='contentDetails',
        playlistId=playlistid,
        maxResults = 50,
        pageToken=nextpagetoken
      )
      response = request.execute()
      for i in range(len(response['items'])):
        videoids.append(response['items'][i]['contentDetails']['videoId'])

      nextpagetoken = response.get('nextPageToken')

  return videoids
  

In [ ]:
videIDs = getvideoids(playlistid)
len(videIDs)


### function to get a specific channel's video details based on video ID.

In [ ]:
def getvideodetails(videIDs):
    videoStats = []
    for i in range(0,len(videIDs),50):
        request = youtube.videos().list(
            part = 'snippet,statistics',
            id = ','.join(videIDs[i:i+50])
        )
        response = request.execute()
        for video in response['items']:
            videoinfo = dict(
                Title = video['snippet']['title'],
                PublishedDate = video['snippet']['publishedAt'],
                ViewCount = video['statistics']['viewCount'],
                LikeCount = video['statistics']['likeCount'],
                # DislikeCount = video['statistics']['dislikeCount'],
                CommentCount = video['statistics']['commentCount']
            )
            videoStats.append(videoinfo)
    return videoStats

In [ ]:
videodetails = getvideodetails(videIDs)
videodata = pd.DataFrame(videodetails)
videodata

In [ ]:
videodata['PublishedDate'] = pd.to_datetime(videodata['PublishedDate']).dt.date
videodata['ViewCount'] = pd.to_numeric(videodata['ViewCount'])
videodata['LikeCount'] = pd.to_numeric(videodata['LikeCount'])
videodata['CommentCount'] = pd.to_numeric(videodata['CommentCount'])
videodata

In [ ]:
top10videos = videodata.sort_values(by='ViewCount', ascending=False).head(10)
top10videos

In [ ]:
ax1 = sns.barplot(x='ViewCount',y='Title',data=top10videos)

In [ ]:
videodata

In [ ]:
videodata['Month'] = pd.to_datetime(videodata['PublishedDate']).dt.strftime("%b")

In [ ]:
videodata

In [ ]:
vidoespermonth = videodata.groupby('Month',as_index=False).size()
vidoespermonth

In [ ]:
sortorder = ['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec']

In [ ]:
vidoespermonth.index = pd.CategoricalIndex(vidoespermonth['Month'],categories=sortorder, ordered=True)
vidoespermonth = vidoespermonth.sort_index()
vidoespermonth

In [ ]:
ax2 = sns.barplot(x='Month',y ='size',data=vidoespermonth)

In [ ]:
videodata